# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [16]:
# imports

import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import requests


In [ ]:
# constants

load_dotenv(override=True)
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
OLLAMA_BASE_URL = "http://localhost:11434/v1"
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please be sure to add your key to the .env file, and save the file! Or you can skip the next 2 cells if you don't want to use Gemini")
elif not api_key.startswith("AIz"):
    print("An API key was found, but it doesn't start AIz")
else:
    print("API key found and looks good so far!")



In [41]:
# set up environment

ollama = OpenAI(
    base_url=OLLAMA_BASE_URL, 
    api_key='ollama',
    timeout=60
    )
openai_client = OpenAI(
    api_key=api_key,
    base_url="https://openrouter.ai/api/v1",
    default_headers={
        "HTTP-Referer": "http://localhost:8888",
        "X-Title": "llm_engineering_course"
    })
SYSTEM_PROMPT = """
You are a precise book-summary assistant.

Rules:
- Use ONLY the provided public description, title, Date published and authors.
- Do NOT add facts not present in the description.
- Do NOT include release/promotional lines unless central to the book's ideas.
- Output must follow the exact format below.
- Keep total output under 170 words.

Exact output format:
What it's about: <one sentence>
Author(s): <authors>
Date published: <date>

Key ideas:
- <idea 1>
- <idea 2>
- <idea 3>
- <idea 4>
- <idea 5>

Who it's for: <one line>
""".strip()


def build_user_prompt(meta: dict) -> str:
    authors = ", ".join(meta.get("authors") or []) or "Unknown"
    return f"""
Title: {meta.get("title", "Unknown")}
Author(s): {authors},
Date published: {meta.get("publishedDate", "Unknown")}

Public description:
{meta.get("description", "")}

Now follow the exact output format from the system message.
Do not write paragraphs beyond that format.
""".strip()

In [ ]:
# define function to look up books




def lookup_book_google(title: str, author: str, max_results: int = 3):
    """
    Look up a book on Google Books API using title + author,
    then return normalized metadata for the best match.

    Parameters:
        title (str): Book title to search for.
        author (str): Author name to search for.
        max_results (int): How many results to request from API.

    Returns:
        dict | None:
            Normalized metadata dict for best match, or None if no match found.
    """

    # Build Google Books query in the same style as:
    # intitle:<title> inauthor:<author>
    query = f"intitle:{title} inauthor:{author}"

    # Endpoint and query params (requests handles URL encoding for us).
    url = "https://www.googleapis.com/books/v1/volumes"
    params = {"q": query, "maxResults": max_results}

    # Make HTTP request and fail fast for HTTP errors.
    response = requests.get(url, params=params, timeout=20)
    response.raise_for_status()

    # Parse JSON payload from Google Books API.
    data = response.json()

    # If no results were found, return None.
    items = data.get("items", [])
    if not items:
        return None

    # Pick the first result as "best" for now (will improve ranking later).
    best = items[0].get("volumeInfo", {})

    # Return only the fields we care about, with safe defaults.
    return {
        "title": best.get("title"),
        "authors": best.get("authors", []),
        "publishedDate": best.get("publishedDate"),
        "categories": best.get("categories", []),
        "description": best.get("description"),   # key text for summarization
        "pageCount": best.get("pageCount"),
        "previewLink": best.get("previewLink"),
    }

In [48]:
# llm call with openai

def llm_with_gpt(user_prompt: str, stream=False) -> str:
    response = openai_client.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0,
        stream=stream
    )

    if stream:
        text = ""
        display_handle = display(Markdown(""), display_id=True)
        for chunk in response:
            text += chunk.choices[0].delta.content or ""
            update_display(Markdown(text), display_id=display_handle.display_id)
        return text

    return response.choices[0].message.content

In [46]:
#llm call with llama

def llm_with_llama(user_prompt: str, stream=False) -> str:
    response = ollama.chat.completions.create(
        model=MODEL_LLAMA,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt}
            ],
        stream=stream
    )

    if stream:
        text = ""
        display_handle = display(Markdown(""), display_id=True)
        for chunk in response:
            text += chunk.choices[0].delta.content or ''
            update_display(Markdown(text), display_id=display_handle.display_id)
        return text
    return response.choices[0].message.content

In [49]:
#function to summarize book
def summarize_book(meta: dict, llm_callable, stream=False):
    """
    Summarize a book using metadata from a public source.

    Parameters:
        meta (dict): Book metadata (title, authors, description, etc.)
        llm_callable (callable): Function that takes a prompt string and returns model output.
                                 Example signature: llm_callable(prompt: str) -> str

    Returns:
        str: Structured summary text from the LLM, or a fallback message.
    """

    # If metadata is missing or no description is available, return a safe fallback.
    if not meta or not meta.get("description"):
        return "I couldn't find a public description for this book from the source used."


    # Build the prompt and explicitly constrain the model to the provided description.
    prompt = build_user_prompt(meta)

    # Call the LLM wrapper function and return its output.
    output = llm_callable(prompt, stream=stream)
    return output

In [ ]:
# lookup book with google books api
meta = lookup_book_google("Atomic Habits", "James Clear")
print(meta)

In [ ]:
# Get gpt-4o-mini to answer
summary = summarize_book(meta, llm_with_gpt)
print(summary)

In [ ]:
# Get Llama 3.2 to answer
summary = summarize_book(meta, llm_with_llama)
print(summary)

In [ ]:
#llm call with llama with stream
summary = summarize_book(meta, llm_with_llama,True)
# print(summary)

In [ ]:
#llm call with gpt with stream
summary = summarize_book(meta, llm_with_gpt,True)